Speed Distribution: Each car's constant speed is modelled as a uniform distribution S ~ Uniform[1, 2]

Cars go in an assigned lane based on their speed:
<br>-Slow Lane: S &le; a
<br>-Fast Lane: S &ge; a

Assumptions:
We assume each car overtakes at most one other car

In [ ]:
# PDF of S
import sympy as sp
s = sp.symbols('s')
f = sp.Piecewise((1, (s >= 1) & (s <= 2)), (0, True))

#print(sp.integrate(f, (s, 1, 2)) == 1)

Distance Loss Per Overtake:

Slow Lane (S &le; a):
-Slower car decelerates to 0 then re-accelerates to <em>y</em> at unit rate:<code>L<sub>slow</sub>(y) = y<sup>2</sup></code><br>
Fast Lane (S &ge; a):
-Slower car decelerates to 0 then re-accelerates to <em>y</em> at unit rate:<code>L<sub>fast</sub>(y) = (y-a)<sup>2</sup></code><br>
Weight each overtake by relative speed (x - y) and integrate over 1 &le; y &lt; x &le; 2:
<br>
<code>
I<sub>slow</sub>(a) = &int;<sub>x=1</sub><sup>a</sup> &int;<sub>y=1</sub><sup>x</sup> (x &minus; y) y<sup>2</sup> dy dx,<br>
I<sub>fast</sub>(a) = &int;<sub>x=a</sub><sup>2</sup> &int;<sub>y=a</sub><sup>x</sup> (x &minus; y)(y &minus; a)<sup>2</sup> dy dx,<br>
g(a) = I<sub>slow</sub>(a) + I<sub>fast</sub>(a).
</code>


In [4]:
# Symbolic integration to derive g(a)
a, x, y = sp.symbols('a x y')
I_slow = sp.integrate(sp.integrate((x-y)*y**2, (y, 1, x)), (x, 1, a))
I_fast = sp.integrate(sp.integrate((x-y)*(y-a)**2, (y, a, x)), (x, a, 2))
g = sp.simplify(I_slow + I_fast)
print('g(a) =', g)

g(a) = a**4/6 - 2*a**3/3 + 7*a**2/6 - 13*a/12 + 13/30


Finding the value of a that minimises the expected loss function

Compute the derivative g'(a) and find roots in [1, 2]
Confirm that the second derivative is positive at optimum

In [9]:
dg = sp.diff(g, a)
crit = sp.nroots(dg)
real_crit = [r for r in crit if abs(sp.im(r)) < 1e-8]
float_roots = [float(sp.re(r)) for r in real_crit]
opt = [r for r in float_roots if 1 < r < 2][0]
print('Critical a=', opt)
# second derivative
d2g = sp.diff(dg, a)
print('Minimum Value?', d2g.subs(a, opt).evalf() > 0)

Critical a= 1.2266988257582019
Minimum Value? True


Numeric Check

valuate g(a) near the candidates optimum to verify it is minimal

In [ ]:
import numpy as np
values = [1.2, 1.2266988257582019, 1.25]
for a in values:
    print(f"a = {a}, g(a) = {a**4/6 - 2*a**3/3 + 7*a**2/6 - 13*a/12 + 13/30}")

a = 1.2, g(a) = 0.006933333333333236
a = 1.2266988257582019, g(a) = 0.0067806865234583125
a = 1.25, g(a) = 0.0069010416666668295


Final Answer: 

a = 1.2266988258